In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from scipy.ndimage import zoom
import os    
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
from tqdm import tqdm
import math
from torchinfo import summary


device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
class DinoV2(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.dinov2 = torch.hub.load('facebookresearch/dinov2', 'dinov2_vits14')
        for param in self.dinov2.parameters():
            param.requires_grad = False
    def forward(self, x: torch.Tensor):
        with torch.no_grad():
            features = self.dinov2.forward_features(x.to(device))['x_norm_patchtokens']
        return features.reshape(-1, 16, 16, 384).permute(0,3,1,2)

In [3]:
class conv_block(nn.Module):
    def __init__(self,ch_in,ch_out):
        super(conv_block,self).__init__()
        self.quant = torch.ao.quantization.QuantStub()
        self.dequant = torch.ao.quantization.DeQuantStub()
        self.conv1 = nn.Sequential(
            nn.Conv2d(ch_in, ch_out, kernel_size=3,stride=1,padding=1,bias=False),
            nn.BatchNorm2d(ch_out),
            nn.ReLU(inplace=True),
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(ch_out, ch_out, kernel_size=3,stride=1,padding=1,bias=False),
            nn.BatchNorm2d(ch_out),
            nn.ReLU(inplace=True),
        )


    def forward(self,x):
        x = self.quant(x)
        x = self.conv1(x)
        x = self.dequant(x)
        x = self.quant(x)
        x = self.conv2(x)
        x = self.dequant(x)
        return x
    
class conv_block_to_16(nn.Module):
    def __init__(self,ch_in,ch_out):
        super(conv_block_to_16,self).__init__()
        self.quant = torch.ao.quantization.QuantStub()
        self.dequant = torch.ao.quantization.DeQuantStub()
        self.conv1 = nn.Sequential(
            nn.Conv2d(ch_in, ch_in, kernel_size=15,stride=1,padding=1,bias=False),
            nn.BatchNorm2d(ch_in),
            nn.ReLU(inplace=True)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(ch_in, ch_out, kernel_size=3,stride=1,padding=1,bias=False),
            nn.BatchNorm2d(ch_out),
            nn.ReLU(inplace=True)
        )


    def forward(self,x):
        x = self.quant(x)
        x = self.conv1(x)
        x = self.dequant(x)
        x = self.quant(x)
        x = self.conv2(x)
        x = self.dequant(x)
        return x


In [4]:
class U_Net_Student(nn.Module):
    def __init__(self,img_ch=3,output_ch=3):
        super(U_Net_Student,self).__init__()
        self.Maxpool = nn.MaxPool2d(kernel_size=2,stride=2)
        self.Conv1 = conv_block(ch_in=img_ch,ch_out=128)
        self.Conv2 = conv_block(ch_in=128,ch_out=128)
        self.Conv3 = conv_block(ch_in=128,ch_out=384)
        self.Conv4 = conv_block(ch_in=384,ch_out=384)
        self.Conv5 = conv_block(ch_in=384,ch_out=384)
        self.tr = nn.ConvTranspose2d(384, 768, 5, padding=1)
#         self.Conv5 = conv_block_to_16(ch_in=384,ch_out=384*2*2)

    def forward(self,x):
        # encoding path
        x1 = self.Conv1(x)

        x2 = self.Maxpool(x1)
        x2 = self.Conv2(x2)
        
        x3 = self.Maxpool(x2)
        x3 = self.Conv3(x3)

        x4 = self.Maxpool(x3)
        x4 = self.Conv4(x4)
        
        x5 = self.Maxpool(x4)
        x5 = self.Conv5(x5)
        
        x6 = self.tr(x5)
        return x6



In [5]:
transform_img_teacher = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ]
)

transform_img_normal = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ]
)

batch_size = 16
num_workers = 0

In [6]:
import os
from PIL import Image
from torch.utils.data import Dataset
import numpy as np
def all_image(root, train_testgood_testfail=None):
    all_path = []
    for path, subdirs, files in os.walk(root):
        for name in files:
            if ".jpg" in name.lower() or ".png" in name.lower() or ".jpeg" in name.lower():
                if train_testgood_testfail=="train":
                    if "train" in os.path.join(path, name) or "Normal" in os.path.join(path, name):
                        all_path.append(os.path.join(path, name))
                elif train_testgood_testfail=="test_good":
                    if ("test" in os.path.join(path, name) and "good" in os.path.join(path, name)) or ("test" in os.path.join(path, name) and "validation" in os.path.join(path, name) and "good" in os.path.join(path, name)):
                        all_path.append(os.path.join(path, name))
                elif train_testgood_testfail=="test_fail":
                    if "test" in os.path.join(path, name) and "good" not in os.path.join(path, name) or "Anomaly" in os.path.join(path, name):
                        all_path.append(os.path.join(path, name))
    return all_path

class GetDataset(Dataset):
    def __init__(self, image_dir, transform_teacher=None, transform_normal=None, train_testgood_testfail=None):
        self.image_dir = image_dir
        self.transform_teacher = transform_teacher
        self.transform_normal = transform_normal
        self.images = all_image(root=image_dir, train_testgood_testfail=train_testgood_testfail) # list all tye files in directory(folder)

    def __len__(self): # check length of that dataset
        return len(self.images)

    def __getitem__(self, index): # take as input some index
        img_path = self.images[index]
        image = Image.open(img_path).convert("RGB")
        image_teacher_student = self.transform_teacher(image)
        image_normal = self.transform_normal(image)
        image_aug = self.random_transform(image)

        return image_teacher_student, image_normal, image_aug
    
    def random_transform(self, image):
        random_param = random.uniform(0.8, 1.2)
        random_class = random.randint(1, 3)
        if random_class == 1:
            tf = transforms.Compose(
                    [
                        transforms.Resize((256, 256)),
                        transforms.CenterCrop(224),
                        transforms.RandomAutocontrast(0.1),
                        transforms.RandomEqualize(0.1),
                        transforms.ColorJitter(brightness=random_param, contrast=0, saturation=0, hue=0),
                        transforms.ToTensor(),
                        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
                    ]
                )
        elif random_class == 2:
            tf = transforms.Compose(
                    [
                        transforms.Resize((256, 256)),
                        transforms.CenterCrop(224),
                        transforms.RandomAutocontrast(0.1),
                        transforms.RandomEqualize(0.1),
                        transforms.ColorJitter(brightness=0, contrast=random_param, saturation=0, hue=0),
                        transforms.ToTensor(),
                        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
                    ]
                )
        else:
            tf = transforms.Compose(
                    [
                        transforms.Resize((256, 256)),
                        transforms.CenterCrop(224),
                        transforms.RandomAutocontrast(0.1),
                        transforms.RandomEqualize(0.1),
                        transforms.ColorJitter(brightness=0, contrast=0, saturation=random_param, hue=0),
                        transforms.ToTensor(),
                        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
                    ]
                )
        return tf(image)
    
image_train_dir = r"D:\Delete\MV\splicing_connectors"
image_test_dir = r"D:\Delete\MV\splicing_connectors"
train_dataset = GetDataset(image_dir=image_train_dir, transform_teacher=transform_img_teacher, transform_normal=transform_img_normal, train_testgood_testfail="train")
test_good_dataset = GetDataset(image_dir=image_test_dir, transform_teacher=transform_img_teacher, transform_normal=transform_img_normal, train_testgood_testfail="test_good")
test_fail_dataset = GetDataset(image_dir=image_test_dir, transform_teacher=transform_img_teacher, transform_normal=transform_img_normal, train_testgood_testfail="test_fail")
    
print(len(train_dataset))
print(len(test_good_dataset))
print(len(test_fail_dataset))
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, num_workers = num_workers,shuffle=True, pin_memory=True)
test_good_loader = torch.utils.data.DataLoader(test_good_dataset, batch_size=8, num_workers = num_workers,shuffle=True, pin_memory=True)
test_fail_loader = torch.utils.data.DataLoader(test_fail_dataset, batch_size=8, num_workers = num_workers,shuffle=True, pin_memory=True)

360
119
193


In [7]:
import matplotlib.pyplot as plt
import random
import math

model_teacher = DinoV2().to(device).eval()
model_student = U_Net_Student().to(device).train()
IMAGE_HEIGHT = 224
IMAGE_WIDTH = 224
print(summary(model=model_student,
        input_size=(1, 3, IMAGE_HEIGHT, IMAGE_WIDTH),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]))
# print(model_student)
#test quantize
# model_fp32 = model_student.eval()
# model_fp32.qconfig = torch.ao.quantization.get_default_qat_qconfig('x86')
# model_fp32_fused = torch.ao.quantization.fuse_modules(model_fp32,
#     [
#          ['Conv1.conv1.0', 'Conv1.conv1.1', 'Conv1.conv1.2'],
#          ['Conv1.conv2.0', 'Conv1.conv2.1', 'Conv1.conv2.2'],
        
#          ['Conv2.conv1.0', 'Conv2.conv1.1', 'Conv2.conv1.2'],
#          ['Conv2.conv2.0', 'Conv2.conv2.1', 'Conv2.conv2.2'],
        
#          ['Conv3.conv1.0', 'Conv3.conv1.1', 'Conv3.conv1.2'],
#          ['Conv3.conv2.0', 'Conv3.conv2.1', 'Conv3.conv2.2'],
        
#         ['Conv4.conv1.0', 'Conv4.conv1.1', 'Conv4.conv1.2'],
#          ['Conv4.conv2.0', 'Conv4.conv2.1', 'Conv4.conv2.2'],
        
#         ['Conv5.conv1.0', 'Conv5.conv1.1', 'Conv5.conv1.2'],
#          ['Conv5.conv2.0', 'Conv5.conv2.1', 'Conv5.conv2.2'],
#     ])
# model_student = torch.ao.quantization.prepare_qat(model_fp32_fused.to(device).train())
epochs = 500


optimizer_student = torch.optim.AdamW(model_student.parameters(), lr=1e-4, weight_decay=1e-5)

loop_train = tqdm(enumerate(train_loader),total=len(train_loader))
all_chanel_of_image = []

for batch_idx, (image_teacher_student, img_normal, image_augment) in loop_train:
    with torch.no_grad():
        feature_out_teacher = model_teacher(image_teacher_student)
    all_chanel_of_image.append(feature_out_teacher)
all_chanel_of_image = torch.cat(all_chanel_of_image, dim=0)
mean_all_chanel = torch.mean(all_chanel_of_image, dim=(0,2,3)).unsqueeze(0).unsqueeze(2).unsqueeze(3)
std_all_chanel = torch.std(all_chanel_of_image, dim=(0,2,3)).unsqueeze(0).unsqueeze(2).unsqueeze(3)
k = 112
del all_chanel_of_image

for epoch in range(epochs):
    loss_overall = []
    model_student.train()
    score_train = []
    self_diff_train = []
    quatile_train = []
    loop_train = tqdm(enumerate(train_loader),total=len(train_loader))
    for batch_idx, (image_teacher_student, img_normal, image_augment) in loop_train:
        image_teacher_student = image_teacher_student.to(device)
        img_normal = img_normal.to(device)
        image_augment = image_augment.to(device)
        
        optimizer_student.zero_grad()
        feature_student = model_student(image_teacher_student)
        feature_student_aug = model_student(image_augment)
        
        with torch.no_grad():
            feature_out_teacher_nomal = model_teacher(image_teacher_student)
            feature_out_teacher_augment = model_teacher(image_augment)
            feature_out_teacher_normal_normalize = (feature_out_teacher_nomal - mean_all_chanel)/std_all_chanel
            feature_out_teacher_augment_normalize = (feature_out_teacher_augment - mean_all_chanel)/std_all_chanel
        
        loss_student_teacher = torch.nn.functional.mse_loss(feature_student[:,:384,:,:], feature_out_teacher_normal_normalize)
        loss_student_ae = torch.nn.functional.mse_loss(feature_student_aug[:,384:,:,:], feature_out_teacher_augment_normalize)
        loss_all = loss_student_teacher + loss_student_ae
        loss_all.backward()
        optimizer_student.step()
        
        global_map = torch.sum((feature_student[:,:384,:,:] - feature_out_teacher_normal_normalize)**2, dim=1)/384
        local_map = torch.sum((feature_student[:,384:,:,:] - feature_out_teacher_normal_normalize)**2, dim=1)/384
        diff_self = torch.sum((feature_student[:,384:,:,:] - feature_student[:,:384,:,:])**2, dim=1)/384
        value_global, _ = torch.topk(global_map.flatten(start_dim=1), k, dim=1)
        value_map, _ = torch.topk(local_map.flatten(start_dim=1), k, dim=1)
            
        value = (value_global + value_map)/2
        score_train.append(torch.mean(value))
        loss_overall.append(loss_all.detach().item())
        
    score_good = []
    self_diff_good = []
    quatile_good = []
    for image_teacher_student, img_normal, _ in test_good_loader:
        model_student.eval()
        image_teacher_student = image_teacher_student.to(device)
        img_normal = img_normal.to(device)
        with torch.no_grad():
            feature_student = model_student(image_teacher_student)
            feature_out_teacher = model_teacher(image_teacher_student)
        feature_out_teacher_normalize = (feature_out_teacher - mean_all_chanel) / std_all_chanel
        global_map = torch.sum((feature_student[:,:384,:,:] - feature_out_teacher_normalize)**2, dim=1)/384
        local_map = torch.sum((feature_student[:,384:,:,:] - feature_out_teacher_normalize)**2, dim=1)/384
        combine_map = (global_map + local_map)/2
        value_global, _ = torch.topk(global_map.flatten(start_dim=1), k, dim=1)
        value_map, _ = torch.topk(local_map.flatten(start_dim=1), k, dim=1)
        value = (value_global + value_map)/2
        score_good.append(torch.mean(value))
    
    score_fail = []
    self_diff_fail = []
    quatile_fail = []
    for image_teacher_student, img_normal, _ in test_fail_loader:
        model_student.eval()
        image_teacher_student = image_teacher_student.to(device)
        img_normal = img_normal.to(device)
        with torch.no_grad():
            feature_student = model_student(image_teacher_student)
            feature_out_teacher = model_teacher(image_teacher_student)
        feature_out_teacher_normalize = (feature_out_teacher - mean_all_chanel) / std_all_chanel
        global_map = torch.sum((feature_student[:,:384,:,:] - feature_out_teacher_normalize)**2, dim=1)/384
        local_map = torch.sum((feature_student[:,384:,:,:] - feature_out_teacher_normalize)**2, dim=1)/384
        combine_map = (global_map + local_map)/2
        diff_self = torch.sum((feature_student[:,384:,:,:] - feature_student[:,:384,:,:])**2, dim=1)/384
        
        global_map = torch.sum((feature_student[:,:384,:,:] - feature_out_teacher_normalize)**2, dim=1)/384
        local_map = torch.sum((feature_student[:,384:,:,:] - feature_out_teacher_normalize)**2, dim=1)/384

        value_global, _ = torch.topk(global_map.flatten(start_dim=1), k, dim=1)
        value_map, _ = torch.topk(local_map.flatten(start_dim=1), k, dim=1)
        value = (value_global + value_map)/2
        score_fail.append(torch.mean(value))
        
#     f, axarr = plt.subplots(1,3, figsize=(224, 224))
#     axarr[0].imshow(img_normal[-1].permute(1,2,0).cpu().detach().numpy())
#     axarr[1].imshow(img_normal[-1].permute(1,2,0).cpu().detach().numpy())
#     axarr[1].imshow(zoom(np.squeeze(global_map[-1].cpu().detach().numpy()),zoom=(224/16,224/16)), cmap='jet', alpha=0.4)
#     axarr[2].imshow(img_normal[-1].permute(1,2,0).cpu().detach().numpy())
#     axarr[2].imshow(zoom(np.squeeze(local_map[-1].cpu().detach().numpy()),zoom=(224/16,224/16)), cmap='jet', alpha=0.4)

#     axarr[3].imshow(img_normal[-1].permute(1,2,0).cpu().detach().numpy())
#     axarr[3].imshow(zoom(np.squeeze(combine_map[-1].cpu().detach().numpy()),zoom=(224/16,224/16)), cmap='jet', alpha=0.4)
#     axarr[4].imshow(img_normal[-1].permute(1,2,0).cpu().detach().numpy())
#     axarr[4].imshow(zoom(np.squeeze(diff_self[-1].cpu().detach().numpy()),zoom=(224/16,224/16)), cmap='jet', alpha=0.4)
    plt.show()
    print(f"loss_overall {np.array(loss_overall).mean()}")
    print(f"epoch {epoch+1}")
    
    print()
    
    print(f"score_test_good_mean {torch.Tensor(score_good).mean()}")
    print(f"score_test_good_std {torch.Tensor(score_good).std()}")
    print(f"score_test_good_min {torch.Tensor(score_good).min()}")
    print(f"score_test_good_max {torch.Tensor(score_good).max()}")
    
    print()
    
    print(f"score_test_fail_mean {torch.Tensor(score_fail).mean()}")
    print(f"score_test_fail_std {torch.Tensor(score_fail).std()}")
    print(f"score_test_fail_min {torch.Tensor(score_fail).min()}")
    print(f"score_test_fail_max {torch.Tensor(score_fail).max()}")
    
    print()
    
    print(f"score_train_mean {torch.Tensor(score_train).mean()}")
    print(f"score_train_std {torch.Tensor(score_train).std()}")
    print(f"score_train_min {torch.Tensor(score_train).min()}")
    print(f"score_train_max {torch.Tensor(score_train).max()}")

Using cache found in C:\Users\suppa/.cache\torch\hub\facebookresearch_dinov2_main
xFormers not available
xFormers not available


Layer (type (var_name))                  Input Shape          Output Shape         Param #              Trainable
U_Net_Student (U_Net_Student)            [1, 3, 224, 224]     [1, 768, 16, 16]     --                   True
├─conv_block (Conv1)                     [1, 3, 224, 224]     [1, 128, 224, 224]   --                   True
│    └─QuantStub (quant)                 [1, 3, 224, 224]     [1, 3, 224, 224]     --                   --
│    └─Sequential (conv1)                [1, 3, 224, 224]     [1, 128, 224, 224]   --                   True
│    │    └─Conv2d (0)                   [1, 3, 224, 224]     [1, 128, 224, 224]   3,456                True
│    │    └─BatchNorm2d (1)              [1, 128, 224, 224]   [1, 128, 224, 224]   256                  True
│    │    └─ReLU (2)                     [1, 128, 224, 224]   [1, 128, 224, 224]   --                   --
│    └─DeQuantStub (dequant)             [1, 128, 224, 224]   [1, 128, 224, 224]   --                   --
│    └─QuantStub (qu

100%|██████████████████████████████████████████████████████████████████████████████████| 23/23 [00:33<00:00,  1.45s/it]


loss_overall 1.6465660644614177
epoch 1

score_test_good_mean 0.8786736130714417
score_test_good_std 0.017795030027627945
score_test_good_min 0.8544409871101379
score_test_good_max 0.9194731712341309

score_test_fail_mean 0.9261922240257263
score_test_fail_std 0.033566322177648544
score_test_fail_min 0.8662306070327759
score_test_fail_max 1.0075749158859253

score_train_mean 1.0059162378311157
score_train_std 0.15487797558307648
score_train_min 0.8494310975074768
score_train_max 1.4398022890090942


100%|██████████████████████████████████████████████████████████████████████████████████| 23/23 [00:33<00:00,  1.47s/it]


...

loss_overall 0.2255711458299471
epoch 450

score_test_good_mean 0.5651596188545227
score_test_good_std 0.017643578350543976
score_test_good_min 0.5315408706665039
score_test_good_max 0.5929000377655029

score_test_fail_mean 0.6531358957290649
score_test_fail_std 0.06902870535850525
score_test_fail_min 0.5497384071350098
score_test_fail_max 0.7877494692802429

score_train_mean 0.22348541021347046
score_train_std 0.006282719783484936
score_train_min 0.2116064429283142
score_train_max 0.2363525927066803


100%|██████████████████████████████████████████████████████████████████████████████████| 23/23 [00:33<00:00,  1.45s/it]


loss_overall 0.22386913519838583
epoch 451

score_test_good_mean 0.5467920303344727
score_test_good_std 0.021931149065494537
score_test_good_min 0.5156787037849426
score_test_good_max 0.5991207361221313

score_test_fail_mean 0.654079258441925
score_test_fail_std 0.0631060004234314
score_test_fail_min 0.5856030583381653
score_test_fail_max 0.8904985189437866

score_train_mean 0.22363081574440002
score_train_std 0.005367169622331858
score_train_min 0.21030215919017792
score_train_max 0.23325461149215698


100%|██████████████████████████████████████████████████████████████████████████████████| 23/23 [00:39<00:00,  1.71s/it]


KeyboardInterrupt: 

In [ ]:
# model_student.to('cpu').eval()
# model_int8 = torch.ao.quantization.convert(model_student)
# input_fp32 = torch.randn(1, 3, 224, 224)
# res = model_int8(input_fp32)

In [ ]:
print(res)